In [2]:
import openpyxl
from datetime import date
from datetime import datetime
import requests, bs4
import urllib3
import os

#function for getting stock price and updating the excel
def get_stock_price(stockcode):
    stock_web = requests.get(f'https://sg.finance.yahoo.com/quote/{stockcode}.SI/', verify = False)
    stock_price = bs4.BeautifulSoup(stock_web.text).select('div div span')
    return float(stock_price[13].getText())


def get_dividend(stockcode):
    dividend = requests.get(f'https://www.dividends.sg/view/{stockcode}', verify = False)
    pay_date = bs4.BeautifulSoup(dividend.text).select('td.hidden-xs')
    amount = bs4.BeautifulSoup(dividend.text).select('table td')
    
    #counting number of dividend pay dates that is after the date that report was last ran
    count = 0
    for x in pay_date:
        if x.getText()[:-2] > str(max(datelist)): #if pay date is after the last date report was done
            count += 1        
    sum_of_dividend = 0
    n = 3 #first dividend amoutn starts at index 3
    while count != 0:
        sum_of_dividend += float(amount[n].getText()[3:-2]) 
        count -= 1
        n += 4 #each dividend amount is in intervals of 4
    return sum_of_dividend

## code starts here ##

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#finding today's date and the date that report was last ran, then format it to what i want.

files = os.listdir(r"C:\Users\314798\Desktop\test folder") #list of file names by alphabetical order

datelist = []
for x in files:
    if len(x) > 20:
        datelist.append(datetime.strptime(x[22:-5], '%d-%m-%Y'))

today = date.today().strftime('%d-%m-%Y')
previousdate = max(datelist).strftime('%d-%m-%Y')

wb = openpyxl.load_workbook(f"C:\\Users\\314798\\Desktop\\test folder\\dad's portfolio as of {previousdate}.xlsx")
ws = wb.active

#updating today's date on cell A1
ws['A1'] = today


ws['E4'] = get_stock_price('D05')
ws['E5'] = get_stock_price('U11')
ws['E6'] = get_stock_price('O39')
ws['E7'] = get_stock_price('CC3')
ws['E8'] = get_stock_price('Z74')
ws['E9'] = get_stock_price('OV8')
ws['E10'] = get_stock_price('1B0')
ws['E11'] = get_stock_price('ND8U')
ws['E12'] = get_stock_price('S7OU')

ws['G4'] = get_dividend('D05') * 1200
ws['G5'] = get_dividend('U11') * 800
ws['G6'] = get_dividend('O39') * 2500
ws['G7'] = get_dividend('CC3') * 7000
ws['G8'] = get_dividend('Z74') * 5000
ws['G9'] = get_dividend('OV8') * 15000
ws['G10'] = get_dividend('1B0') * 26000
ws['G11'] = get_dividend('ND8U') * 10500
ws['G12'] = get_dividend('S7OU') * 25000

ws['B15'].value += ws['G4'].value
ws['B16'].value += ws['G5'].value
ws['B17'].value += ws['G6'].value
ws['B18'].value += ws['G7'].value
ws['B19'].value += ws['G8'].value
ws['B20'].value += ws['G9'].value
ws['B21'].value += ws['G10'].value
ws['B22'].value += ws['G11'].value
ws['B23'].value += ws['G12'].value

wb.save(f"C:\\Users\\314798\\Desktop\\test folder\\dad's portfolio as of {today}.xlsx")